In [3]:
import numpy as np
import string
from algo_euclid import euclid
from ext_algo_euclid import euclides_extendido_pasos 


In [4]:
def inverso(a,b):
    d, s, t, tabla = euclides_extendido_pasos(a, b)
    if s<0:
        s = s + b
    return s

In [5]:
ALFABETO = string.ascii_uppercase
M = 26

def normalizar(texto: str) -> str:
    return "".join([c for c in texto.upper() if c in ALFABETO])

def inv_matriz_mod_np(K, mod=26):
    """
    Inversa modular de K (j×j) usando Gauss-Jordan con NumPy.
    Usa:
      - euclid(a,b)  -> gcd(a,b)
      - inverso(a,b) -> a^{-1} mod b
    """
    K = np.array(K, dtype=int) % mod
    n = K.shape[0]
    if K.shape[1] != n:
        raise ValueError("K debe ser cuadrada (j×j).")

    A = K.copy()
    I = np.eye(n, dtype=int)

    for col in range(n):
        # buscar pivote invertible: gcd(pivote, mod) = 1
        piv = None
        for r in range(col, n):
            if euclid(int(A[r, col]), mod) == 1:
                piv = r
                break
        if piv is None:
            raise ValueError("La matriz no es invertible mod 26.")

        # intercambio de filas
        if piv != col:
            A[[col, piv]] = A[[piv, col]]
            I[[col, piv]] = I[[piv, col]]

        # normalizar pivote a 1
        inv_p = inverso(int(A[col, col]), mod)
        A[col] = (A[col] * inv_p) % mod
        I[col] = (I[col] * inv_p) % mod

        # eliminación del resto de la columna
        for r in range(n):
            if r == col:
                continue
            factor = int(A[r, col])
            if factor != 0:
                A[r] = (A[r] - factor * A[col]) % mod
                I[r] = (I[r] - factor * I[col]) % mod

    return I % mod

def hill_cifrar(texto: str, K):
    K = np.array(K, dtype=int) % M
    j = K.shape[0]

    P = normalizar(texto)
    P += "X" * ((-len(P)) % j)   # padding a múltiplo de j

    nums = np.array([ord(c) - 65 for c in P], dtype=int)
    bloques = nums.reshape(-1, j)      # vectores fila
    cif = (bloques @ K) % M            # c = mK

    return "".join(chr(x + 65) for x in cif.reshape(-1))

def hill_descifrar(texto_cifrado: str, K):
    K = np.array(K, dtype=int) % M
    j = K.shape[0]

    C = normalizar(texto_cifrado)
    if len(C) % j != 0:
        raise ValueError("Longitud del cifrado no es múltiplo de j.")

    K_inv = inv_matriz_mod_np(K, M)

    nums = np.array([ord(c) - 65 for c in C], dtype=int)
    bloques = nums.reshape(-1, j)
    pla = (bloques @ K_inv) % M         # m = cK^{-1}

    return "".join(chr(x + 65) for x in pla.reshape(-1))


In [6]:
K = [[2,5],
     [3,8]]

print(hill_cifrar("HILL", K))      # MVDN
print(hill_descifrar("MVDN", K))   # HILL


MVDN
HILL
